# Previsão de Votação de Deputados

Modelos preditivos de regressão utilizando a biblioteca [scikit learn](http://scikit-learn.org/stable/index.html) para a predição dos votos de deputados federais considerando dados das últimas eleições.

Tutorial utilizado como base [Regularized Linear Models](https://www.kaggle.com/apapiu/regularized-linear-models)


In [223]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import graphviz
from sklearn import preprocessing
from sklearn import tree
from sklearn.preprocessing import OneHotEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from statistics import mean

%matplotlib inline

In [224]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [225]:
train.shape

(7622, 24)

In [226]:
train.isnull().sum()
test.isnull().sum()

ano                                      0
sequencial_candidato                     0
nome                                     0
uf                                       0
partido                                  0
quantidade_doacoes                       0
quantidade_doadores                      0
total_receita                            0
media_receita                            0
recursos_de_outros_candidatos.comites    0
recursos_de_pessoas_fisicas              0
recursos_de_pessoas_juridicas            0
recursos_proprios                        0
recursos_de_partido_politico             0
quantidade_despesas                      0
quantidade_fornecedores                  0
total_despesa                            0
media_despesa                            0
cargo                                    0
sexo                                     0
grau                                     0
estado_civil                             0
ocupacao                                 0
dtype: int6

In [227]:
x_train = train.loc[:,'partido':'ocupacao']
x_test = test.loc[:,'partido':'ocupacao']
print('\nFormato do dataset X_train:',X_train.shape, '\n')
print('\nFormato do dataset X_test:',X_test.shape, '\n')
x_train.head(3)


Formato do dataset X_train: (7622, 19) 


Formato do dataset X_test: (4592, 19) 



,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao
0,PT,6,6,16600.0,2766.67,0.0,9000.0,6300.0,1300.0,0.0,14,14,16583.60,1184.54,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,CASADO(A),VEREADOR
1,PT,13,13,22826.0,1755.85,6625.0,15000.0,1000.0,201.0,0.0,24,23,20325.99,846.92,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),SERVIDOR PÚBLICO ESTADUAL
2,PT,17,16,158120.8,9301.22,2250.0,34150.0,62220.8,59500.0,0.0,123,108,146011.70,1187.09,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,VIÚVO(A),PEDAGOGO


In [228]:
y_train = pd.core.series.Series(train['situacao']=='eleito', dtype='int64')
y_train

0       0
1       0
2       1
3       0
4       1
       ..
7617    0
7618    0
7619    0
7620    0
7621    0
Name: situacao, Length: 7622, dtype: int64

Porcentagem de candidatos não eleitos

In [229]:
nao_eleito = 6596/len(train) * 100
print(nao_eleito)

86.53896615061663


Porcentagem de candidatos eleitos

In [230]:
eleitos = 100 - nao_eleito
print(eleitos)

13.46103384938337


In [231]:
y_train = pd.core.series.Series(train['situacao']=='eleito', dtype='int64')
y_train

0       0
1       0
2       1
3       0
4       1
       ..
7617    0
7618    0
7619    0
7620    0
7621    0
Name: situacao, Length: 7622, dtype: int64

In [232]:
numeric_features = x_train.dtypes[x_train.dtypes != "object"].index
numeric_features

Index(['quantidade_doacoes', 'quantidade_doadores', 'total_receita',
       'media_receita', 'recursos_de_outros_candidatos.comites',
       'recursos_de_pessoas_fisicas', 'recursos_de_pessoas_juridicas',
       'recursos_proprios', 'recursos_de_partido_politico',
       'quantidade_despesas', 'quantidade_fornecedores', 'total_despesa',
       'media_despesa'],
      dtype='object')

In [233]:
x_train[numeric_features] = np.log1p(x_train[numeric_features])
x_test[numeric_features] = np.log1p(x_test[numeric_features])
x_train.head(3)

,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao
0,PT,1.945910,1.945910,9.717218,7.925761,0.000000,9.105091,8.748464,7.170888,0.0,2.708050,2.708050,9.716230,7.077954,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,CASADO(A),VEREADOR
1,PT,2.639057,2.639057,10.035699,7.471278,8.798757,9.615872,6.908755,5.308268,0.0,3.218876,3.178054,9.919705,6.742786,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),SERVIDOR PÚBLICO ESTADUAL
2,PT,2.890372,2.833213,11.971121,9.138008,7.719130,10.438547,11.038461,10.993748,0.0,4.820282,4.691348,11.891449,7.080102,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,VIÚVO(A),PEDAGOGO


In [234]:
categorical_features = x_train.dtypes[x_train.dtypes == "object"].index
categorical_features

Index(['partido', 'cargo', 'sexo', 'grau', 'estado_civil', 'ocupacao'], dtype='object')

In [235]:
min_max_scaler = preprocessing.MinMaxScaler()
x_train[numeric_features] = min_max_scaler.fit_transform(x_train[numeric_features])
x_test[numeric_features] = min_max_scaler.fit_transform(x_test[numeric_features])
x_train.head(3)

,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao
0,PT,0.153520,0.157133,0.624727,0.603972,0.000000,0.643280,0.584328,0.461896,0.0,0.236756,0.241651,0.629989,0.539366,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,CASADO(A),VEREADOR
1,PT,0.238463,0.244074,0.645203,0.569339,0.582155,0.679367,0.461450,0.341920,0.0,0.296779,0.298020,0.643183,0.513825,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),SERVIDOR PÚBLICO ESTADUAL
2,PT,0.269260,0.268426,0.769633,0.696350,0.510723,0.737490,0.737282,0.708136,0.0,0.484948,0.479512,0.771028,0.539530,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,VIÚVO(A),PEDAGOGO


In [236]:
x_train.shape, x_test.shape

((7622, 19), (4592, 19))

#### 1. Há desbalanceamento das classes (isto é, uma classe tem muito mais instâncias que outra)? Em que proporção? Quais efeitos colaterais o desbalanceamento de classes pode causar no classificador? Como você poderia tratar isso? (10 pt.)

In [237]:
train['situacao'].value_counts()

nao_eleito    6596
eleito        1026
Name: situacao, dtype: int64

In [238]:
onehotencode = OneHotEncoder(x_train[categorical_features], use_cat_names=True)
#x_train = onehotencode.fit(x)
dummies = onehotencode.fit_transform(x_train[categorical_features])
variaveis_dummies = list(onehotencode.get_feature_names())
print('Total de variáveis dummy:', len(variaveis_dummies))
variaveis_dummies
train_ohe_test = onehotencode.fit(x_test)

Total de variáveis dummy: 222


c:\Users\rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
c:\Users\rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
c:\Users\rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
c:\Users\rayan\AppData\Local\Programs\Python\Python310\lib\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.i

In [239]:
dummies = pd.DataFrame(dummies, columns = variaveis_dummies)
print(dummies.shape)
dummies.head(3)

(7622, 222)


,partido_PT,partido_PRONA,partido_PCO,partido_PPS,partido_PAN,partido_PSOL,partido_PSTU,partido_PSDC,partido_PDT,partido_PL,...,ocupacao_SERVIDOR DA JUSTIÇA ELEITORAL,ocupacao_FRENTISTA,ocupacao_COMUNICÓLOGO,"ocupacao_ESTIVADOR, CARREGADOR E ASSEMELHADOS",ocupacao_SECURITÁRIO,ocupacao_FAXINEIRO,ocupacao_MODELO,ocupacao_ASTRÔNOMO,ocupacao_FISCAL DE TRANSPORTE COLETIVO,ocupacao_FONOAUDIÓLOGO
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [246]:
x = pd.concat([x_train[numeric_features], dummies], axis=1)
print(x_train.shape)
x_train.head(3)

(5716, 235)


,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,...,ocupacao_SERVIDOR DA JUSTIÇA ELEITORAL,ocupacao_FRENTISTA,ocupacao_COMUNICÓLOGO,"ocupacao_ESTIVADOR, CARREGADOR E ASSEMELHADOS",ocupacao_SECURITÁRIO,ocupacao_FAXINEIRO,ocupacao_MODELO,ocupacao_ASTRÔNOMO,ocupacao_FISCAL DE TRANSPORTE COLETIVO,ocupacao_FONOAUDIÓLOGO
7583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
596,10.0,8.0,68197.1,6819.71,40000.0,3000.0,8000.0,17197.1,0.0,153.0,...,0,0,0,0,0,0,0,0,0,0
3517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [241]:
x_train, x_test, y_train, y_test = train_test_split(x, y)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((5716, 235), (1906, 235), (5716, 1), (1906, 1))

In [242]:
def validacao_cruzada(modelo, x, y, oversampling=False):
    kfold = KFold(n_splits=5)
    acuracias_split = []

    for idx, (idx_treino, idx_validacao) in enumerate(kfold.split(x)):
        x_split_treino = x.iloc[idx_treino, :]
        y_split_treino = y.iloc[idx_treino, :]

        if oversampling:
            sm = SMOTE(random_state=42)
            x_split_treino, y_split_treino = sm.fit_resample(x_split_treino, y_split_treino)

        modelo.fit(x_split_treino, y_split_treino.values.flatten())

        x_split_validacao = x.iloc[idx_validacao, :]
        y_split_validacao = y.iloc[idx_validacao, :]

        predicoes_validacao = modelo.predict(x_split_validacao)

        acuracia_split = accuracy_score(y_split_validacao, predicoes_validacao)

        acuracias_split.append(acuracia_split)

        print(f'Acurácia do split {idx}: {acuracia_split}')

    return acuracias_split

#### 2. Treine: um modelo de regressão logística, uma árvore de decisão, um modelo de adaboost, um modelo de random forest e um modelo de gradient boosting. Tune esses modelos usando validação cruzada e controle overfitting se necessário, considerando as particularidades de cada modelo.  (10 pts.)

In [243]:
modelo_hgb = HistGradientBoostingClassifier()

In [244]:
media_acuracia_sem_smote = mean(validacao_cruzada(modelo_hgb, x_train, y_train, oversampling=False))

Acurácia do split 0: 0.8662587412587412
Acurácia do split 1: 0.8748906386701663
Acurácia do split 2: 0.8538932633420823
Acurácia do split 3: 0.8416447944007
Acurácia do split 4: 0.8661417322834646


In [245]:
media_acuracia_com_smote = mean(validacao_cruzada(modelo_hgb, x_train, y_train, oversampling=True))

ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:

f'Sem smote: {media_acuracia_sem_smote:.02f}, com_smote: {media_acuracia_com_smote:.02f}'

'Sem smote: 0.91, com_smote: 0.91'

# Random Forest

In [ ]:
x_train = train_ohe.drop('partido_PT', axis=1)
x_test = train_ohe_test.drop('partido_PT', axis=1)
y_train = train_ohe['partido_PT']
y_test = train_ohe_test['partido_PT']

AttributeError: 'OneHotEncoder' object has no attribute 'drop'

In [ ]:
mdl = RandomForestClassifier(n_jobs=6, n_estimators=100, random_state=22)
mdl.fit(x_train, y_train)

In [ ]:
predict_ohe = mdl.predict(x_test)

## Erro médio absoluto de 0.05

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, predict_ohe)

In [ ]:
print(classification_report(y_test, predict_ohe))

## Decision Tree

###### -----------

In [ ]:
decision_tree = DecisionTreeClassifier(max_depth=4, min_samples_leaf=0.16, random_state=42)
decision_tree.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

## Acurácia da Árvore de Decisão

In [ ]:
predict_tree = decision_tree.predict(x_test)
accuracy = accuracy_score(y_test, predict_tree)
print('Acurácia da Árvore de Decisão: %.3f%%' % (accuracy * 100))

In [ ]:
print(classification_report(y_test, predict_tree))

# AdaBoost

In [ ]:
decision_tree2 = DecisionTreeClassifier(max_depth=4, random_state=42)
ada_clf = AdaBoostClassifier(decision_tree2, n_estimators=200, learning_rate=0.5)
ada_clf.fit(x_train, y_train)

# Acurácia AdaBoost

In [ ]:
predict_ada = ada_clf.predict(x_test)
accuracy = accuracy_score(y_test, predict_ada)
print('Acurácia AdaBoost: %.3f%%' % (accuracy * 100))

In [ ]:
print(classification_report(y_test, predict_ada))

# Regressão Logística

### Regressão Logística com AdaBoosta

In [ ]:
lr_ada = LogisticRegression(random_state=42, max_iter=1000)
ada_clf2 = AdaBoostClassifier(lr_ada, n_estimators=200, learning_rate=0.5)
ada_clf2.fit(x_train, y_train)

### Acurácia Regressão Logística com AdaBoosta

In [ ]:
predict_lr_ada = ada_clf2.predict(x_test)
accuracy = accuracy_score(y_test, predict_lr_ada)
print('Acurácia Regressão Logística: %.3f%%' % (accuracy * 100))

In [ ]:
print(classification_report(y_test, predict_lr_ada))

## Regressão Logística sem AdaBoost

In [ ]:
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(x_train, y_train)

## Acurácia Regressão Logística

In [ ]:
predict_lr = lr.predict(x_test)
accuracy = accuracy_score(y_test, predict_lr)
print('Acurácia Regressão Logística: %.3f%%' % (accuracy * 100))

In [ ]:
result_mdl = cross_val_score(lr, train_ohe, train_ohe_test, cv=kfold)
print("Validaçaõ: %.3f%%" % (result_mdl.mean() * 100))

In [ ]:
print(classification_report(y_test, predict_lr))

## GradientBoost

In [ ]:
gb_clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.5, random_state=42)
gb_clf.fit(x_train, y_train)

## Acurácia GradientBoost

In [ ]:
predict_gb_clf = gb_clf.predict(x_test)
accuracy = accuracy_score(y_test, predict_gb_clf)
print('Acurácia GradientBoost: %.3f%%' % (accuracy * 100))

In [ ]:
print(classification_report(y_test, predict_gb_clf))

#### 3. Reporte precision, recall e f-measure no treino e validação. Há uma grande diferença de desempenho no treino/validação? Como você avalia os resultados? Justifique sua resposta. (10 pt.)